#### Import needed libraries

In [1]:
#dealing with datasets
import pandas as pd
#operations on datasets
import numpy as np
#building models
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
#creaing viualizations
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
recoveries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/08-22-2020.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/08-22-2020.csv')

In [3]:
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,50013,50190,50433,50655,50810,50886,51039,51280,51350,51405
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,53425,53814,54317,54827,55380,55755,56254,56572,57146,57727
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,95659,96069,96549,97007,97441,97857,98249,98631,98988,99311
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,7602,7633,7669,7699,7756,7806,7821,7875,7919,7983
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,16686,16802,16931,17029,17099,17149,17240,17296,17371,17433


### Preprocessing

###### splitting the dataframes into groups by using groupby() function

In [4]:
confirmed_df=confirmed.groupby(by="Country/Region", as_index=False).sum()
death_df=deaths.groupby(by="Country/Region", as_index=False).sum()
recoveries_df=recoveries.groupby(by="Country/Region", as_index=False).sum()

In [5]:
active_group=pd.DataFrame(columns=[confirmed.columns])
active_group=deaths.copy()


###### dataframe enlargement

In [6]:
for i in range(confirmed_df.shape[0]):
    for j in range(3, confirmed_df.shape[1]):
        active_group.iloc[i,j] = confirmed_df.iloc[i,j]-(recoveries_df.iloc[i,j]+death_df.iloc[i,j])

In [7]:
confirmed.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20
count,270.000000,270.000000,271.000000,271.000000,271.000000,271.000000,271.000000,271.000000,271.000000,271.000000,...,2.710000e+02,2.710000e+02,2.710000e+02,2.710000e+02,2.710000e+02,2.710000e+02,2.710000e+02,2.710000e+02,2.710000e+02,2.710000e+02
mean,20.817839,23.439277,2.047970,2.413284,3.472325,5.291513,7.815498,10.800738,20.583026,22.756458,...,2.855147e+05,2.879002e+05,2.904522e+05,2.929300e+05,2.946553e+05,2.965469e+05,2.981444e+05,2.999478e+05,3.024042e+05,3.051955e+05
std,25.171460,72.529836,27.026899,27.125204,33.769345,46.999290,65.681295,88.495349,217.167147,218.495596,...,1.371642e+06,1.383613e+06,1.397056e+06,1.409380e+06,1.416133e+06,1.428598e+06,1.437439e+06,1.447149e+06,1.459575e+06,1.473547e+06
min,-51.796300,-172.104600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.424864,-18.093125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.980000e+02,7.025000e+02,7.025000e+02,7.135000e+02,7.135000e+02,7.135000e+02,7.135000e+02,7.135000e+02,7.385000e+02,7.385000e+02
50%,22.041450,20.972650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.557000e+03,8.557000e+03,8.557000e+03,8.557000e+03,8.557000e+03,8.557000e+03,8.620000e+03,8.687000e+03,8.934000e+03,8.972000e+03
75%,41.143200,83.380449,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.187690e+05,1.192920e+05,1.205415e+05,1.226440e+05,1.242305e+05,1.253515e+05,1.256680e+05,1.258540e+05,1.260105e+05,1.262875e+05
max,71.706900,178.065000,444.000000,444.000000,549.000000,761.000000,1058.000000,1423.000000,3554.000000,3554.000000,...,1.804505e+07,1.823976e+07,1.846648e+07,1.866534e+07,1.876553e+07,1.899213e+07,1.914260e+07,1.930928e+07,1.951143e+07,1.974047e+07


#### Basic Statistics for Time Series Analysis:
###### Time series analysis comprises methods for analyzing time series data in order to extract meaningful statistics and other characteristics of the data.

In [8]:
base_stats = pd.DataFrame(columns=['Dates','Confirmed','Deaths','Recovered','Active'])
base_stats['Dates'] = confirmed.columns[4:]


In [9]:
base_stats['Confirmed'] = base_stats['Dates'].apply(lambda x: confirmed[x].sum())
base_stats['Recovered'] = base_stats['Dates'].apply(lambda x: recoveries[x].sum())
base_stats['Deaths'] = base_stats['Dates'].apply(lambda x: deaths[x].sum())
base_stats.reset_index(drop=False, inplace=True)
base_stats['Active'] = base_stats['index'].apply(lambda x: (base_stats['Confirmed'][x]-(base_stats['Deaths'][x]+base_stats['Recovered'][x])))

In [10]:
base_stats.head()

,index,Dates,Confirmed,Deaths,Recovered,Active
0,0,1/22/20,555,17,28,510
1,1,1/23/20,654,18,30,606
2,2,1/24/20,941,26,36,879
3,3,1/25/20,1434,42,39,1353
4,4,1/26/20,2118,56,52,2010


#### Recent cases 

In [16]:
recent_stats = go.Figure()
recent_stats.add_trace(go.Treemap(labels = ["Confirmed","Active","Recovered","Deaths"],
                                     parents = ["","Confirmed","Confirmed","Confirmed"],
                                     values = [base_stats['Confirmed'].sum(), base_stats['Active'].sum(), base_stats['Recovered'].sum(), base_stats['Deaths'].sum()],
                                      branchvalues="total", marker_colors = ["#073b4c","#118ab2","#06d6a0","#ef476f"],
                                      textinfo = "label+text+value",
                                      outsidetextfont = {"size": 30, "color": "darkblue"},
                                      marker = {"line": {"width": 2}},
                                        pathbar = {"visible": False}
                                     ))

recent_stats.show()

In [17]:
base_stats_fig = go.Figure()
for column in base_stats.columns.to_list()[2:6]:
    color_dict = {
      "Confirmed": "#ef476f",
      "Deaths": "#118ab2",
      "Recovered": "#073b4c",
      "Active": "#06d6a0"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats[column],
            name = column,
            line = dict(color=color_dict[column]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><i>Count</i>:'+'%{y}',
        )
    )

In [18]:
for column in base_stats.columns.to_list()[2:6]:
    color_dict = {
      "Confirmed": "#0C6583",
      "Deaths": "#24F9C1",
      "Active": "#F47C98",
      "Recovered": "#149ECC"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats['index'].apply(lambda x: (base_stats[column][x-7:x].sum())/7 if x>7 else (base_stats[column][0:x].sum())/7),
            name = column+" 7-day Moving Avg.",
            line = dict(dash="dash", color=color_dict[column]), showlegend=False,
            hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day moving avg.</i>: %{y}'
        )
    )

In [21]:
base_stats_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'All Cases',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True]},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Confirmed',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, True, False, False, False]},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
             dict(label = 'Active',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, True]},
                          {'title': 'Active',
                           'showlegend':True}]),
             dict(label = 'Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, True, False]},
                          {'title': 'Recovered',
                           'showlegend':True}]),
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, True, False, False]},
                          {'title': 'Deaths',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
            showactive=True,
             x=0,
             xanchor="left",
             y=1.25,
             yanchor="top"
        ),
        dict(
        buttons=list(
            [dict(label = 'Linear Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'linear'}},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Log Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'log'}},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
            showactive=True,
             x=0,
             xanchor="left",
             y=1.36,
             yanchor="top"
        )
    ])

base_stats_fig.update_xaxes(showticklabels=False)
base_stats_fig.update_layout( 
    title_text="Basic Statistics for Covid19", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1), paper_bgcolor="mintcream",
                            xaxis_title="Date", yaxis_title="# of Cases")
base_stats_fig.show()